In [17]:
import os
import torch
import json
from io import BytesIO
import numpy as np
import torch.nn as nn
import imageio.v2 as imageio
from tqdm import tqdm, trange
from utils import SphericalNODataset
from neuralop.models import SFNO
import matplotlib.pyplot as plt

In [18]:
device = torch.device('cpu')

In [19]:
data_path = '/Users/reza/Career/DMLab/SURROGATE/Data/psi_web_sample/test'
result_path = '/Users/reza/Career/DMLab/SURROGATE/results/sfno/exp_17'
cfg_path = os.path.join(result_path, 'cfg.json')
with open(cfg_path, 'r') as f:
    cfg = json.load(f)

In [20]:
len(cfg['train_files'])

680

In [21]:
instruments = [
        "kpo_mas_mas_std_0101",
        "mdi_mas_mas_std_0101",
        "hmi_mast_mas_std_0101",
        "hmi_mast_mas_std_0201",
        "hmi_masp_mas_std_0201",
        "mdi_mas_mas_std_0201",
    ]
subdir_paths = sorted(os.listdir(data_path))
cr_paths = [os.path.join(data_path, p) for p in subdir_paths if p.startswith("cr")]
sim_paths = []
for cr_path in cr_paths:
    for instrument in instruments:
        instrument_path = os.path.join(cr_path, instrument)
        if os.path.exists(instrument_path):
            sim_paths.append(instrument_path)

In [22]:
sim_paths = sorted(sim_paths)

In [23]:
sim_paths[-5:]

['/Users/reza/Career/DMLab/SURROGATE/Data/psi_web_sample/test/cr2238/hmi_mast_mas_std_0101',
 '/Users/reza/Career/DMLab/SURROGATE/Data/psi_web_sample/test/cr2238/hmi_mast_mas_std_0201',
 '/Users/reza/Career/DMLab/SURROGATE/Data/psi_web_sample/test/cr2239/hmi_masp_mas_std_0201',
 '/Users/reza/Career/DMLab/SURROGATE/Data/psi_web_sample/test/cr2239/hmi_mast_mas_std_0101',
 '/Users/reza/Career/DMLab/SURROGATE/Data/psi_web_sample/test/cr2239/hmi_mast_mas_std_0201']

In [24]:
dataset = SphericalNODataset(
    sim_paths,
    v_min=cfg["v_min"],
    v_max=cfg["v_max"],
)

  0%|          | 0/21 [00:00<?, ?it/s]

100%|██████████| 21/21 [00:00<00:00, 52.69it/s]


In [25]:
cfg['v_min'], cfg['v_max']

(0.4673037827014923, 1.591561198234558)

In [26]:
model = SFNO(
    n_modes=(cfg['n_modes'], cfg['n_modes']),
    in_channels=1,
    out_channels=139,
    hidden_channels=cfg['hidden_channels'],
    projection_channel_ratio=2,
    factorization="dense",
)
state = torch.load(os.path.join(result_path, 'ckpt', "best_model_state_dict.pt"), map_location=device, weights_only=False)
model.load_state_dict(state)

<All keys matched successfully>

In [27]:
I = -1
instance = dataset[I]
x, y = instance['x'], instance['y']

# cube = torch.randn((1, 141, 64, 64), dtype=torch.float32)
# cube[:, :, 20:40, :] = 1

# x = cube[:, 0, :, :]
# y = cube[0, 1:, :, :]

model.eval()
with torch.no_grad():
    yhats = model(x.unsqueeze(0))

In [28]:
np.save('yhat.npy', yhats.cpu().numpy())

In [29]:
y.shape, yhats.shape

(torch.Size([139, 111, 128]), torch.Size([1, 139, 111, 128]))

In [30]:
cube = y

In [31]:
frames = []
for i in trange(139):
    y = cube[i, :, :]
    yhat = yhats[0, i, :, :]
    # print(cube.shape)
    # y = y.reshape((128, 110))
    # # print(yhat.shape)
    # yhat = yhat.reshape((128, 110))
    # y = y.transpose(1, 0)
    # yhat = yhat.transpose(1, 0)

    error = np.abs(y - yhat)
    error = (error - error.min()) / (error.max() - error.min())
    
    vmin = min(y.min(), yhat.min())
    vmax = max(y.max(), yhat.max())
    fig, axes = plt.subplots(1, 3, figsize=(12, 6))
    cmap = "viridis"

    # # Plot first subplot
    im1 = axes[0].imshow(y, cmap=cmap, vmin=vmin, vmax=vmax)
    axes[0].set_title(f"gt: br002 at {i+1}")

    # Plot second subplot
    im3 = axes[1].imshow(yhat, cmap=cmap, vmin=vmin, vmax=vmax)
    axes[1].set_title(f"pred at {i+1} ")

    cbar = fig.colorbar(im1, ax=axes, orientation="horizontal", fraction=0.1, pad=0.02)

    # Plot second subplot
    im4 = axes[2].imshow(error, cmap="gray")
    axes[2].set_title(f"|pred-gt|")

    # Create a single colorbar
    cbar = fig.colorbar(im4, ax=axes, orientation="vertical", fraction=0.05, pad=0.02)

    # plt.tight_layout()
    buf = BytesIO()
    plt.savefig(buf, format="png")
    frames.append(imageio.imread(buf))
    plt.close()

100%|██████████| 139/139 [00:29<00:00,  4.76it/s]


In [32]:
output_filename = f"v{I}.gif"
fps = 10  # Adjust frames per second as needed

# Create video directly from frames in memory
with imageio.get_writer(output_filename, fps=fps, loop=0) as writer:
    for frame in frames:
        writer.append_data(frame)